In [41]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import pandas as pd
import os
from scipy.stats import norm
import math
import operator

In [113]:
#Importación de datos

ruta_entrada= os.path.join( '.','data')
ruta_salida = os.path.join( '.','output')
datos_valledupar = 'Valledupar'
datos_ideam_valledupar = 'IDEAM'
ruta_serie_valledupar_ideam= os.path.join(ruta_entrada,datos_valledupar,datos_ideam_valledupar+'.csv')
ideam_valledupar= pd.read_csv(ruta_serie_valledupar_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_valledupar = 'NREL'
ruta_serie_valledupar_nrel= os.path.join(ruta_entrada,datos_valledupar,datos_nrel_valledupar+'.csv')
nrel_valledupar= pd.read_csv(ruta_serie_valledupar_nrel, sep = ';',
                encoding='latin-1')

datos_uribia = 'Uribia'
datos_ideam_uribia = 'IDEAM'
ruta_serie_uribia_ideam= os.path.join(ruta_entrada,datos_uribia,datos_ideam_uribia+'.csv')
ideam_uribia= pd.read_csv(ruta_serie_uribia_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_uribia = 'NREL'
ruta_serie_uribia_nrel= os.path.join(ruta_entrada,datos_uribia,datos_nrel_uribia+'.csv')
nrel_uribia= pd.read_csv(ruta_serie_uribia_nrel, sep = ';',
                encoding='latin-1')

datos_sucre= 'Sucre'
datos_ideam_sucre = 'IDEAM'
ruta_serie_sucre_ideam= os.path.join(ruta_entrada,datos_sucre,datos_ideam_sucre+'.csv')
ideam_sucre= pd.read_csv(ruta_serie_sucre_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_sucre = 'NREL'
ruta_serie_sucre_nrel= os.path.join(ruta_entrada,datos_sucre,datos_nrel_sucre+'.csv')
nrel_sucre= pd.read_csv(ruta_serie_sucre_nrel, sep = ';',
                encoding='latin-1')

datos_santamarta = 'SantaMarta'
datos_ideam_santamarta = 'IDEAM'
ruta_serie_santamarta_ideam= os.path.join(ruta_entrada,datos_santamarta,datos_ideam_santamarta+'.csv')
ideam_santamarta= pd.read_csv(ruta_serie_santamarta_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_santamarta = 'NREL'
ruta_serie_santamarta_nrel= os.path.join(ruta_entrada,datos_santamarta,datos_nrel_santamarta+'.csv')
nrel_santamarta= pd.read_csv(ruta_serie_santamarta_nrel, sep = ';',
                encoding='latin-1')

datos_medellin = 'Medellin'
datos_ideam_medellin = 'IDEAM'
ruta_serie_medellin_ideam= os.path.join(ruta_entrada,datos_medellin,datos_ideam_medellin+'.csv')
ideam_medellin= pd.read_csv(ruta_serie_medellin_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_medellin = 'NREL'
ruta_serie_medellin_nrel= os.path.join(ruta_entrada,datos_medellin,datos_nrel_medellin+'.csv')
nrel_medellin= pd.read_csv(ruta_serie_medellin_nrel, sep = ';',
                encoding='latin-1')

datos_carmen = 'Carmendebolivar'
datos_ideam_carmen = 'IDEAM'
ruta_serie_carmen_ideam= os.path.join(ruta_entrada,datos_carmen,datos_ideam_carmen+'.csv')
ideam_carmen= pd.read_csv(ruta_serie_carmen_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_carmen = 'NREL'
ruta_serie_carmen_nrel= os.path.join(ruta_entrada,datos_carmen,datos_nrel_carmen+'.csv')
nrel_carmen= pd.read_csv(ruta_serie_carmen_nrel, sep = ';',
                encoding='latin-1')

datos_cali = 'Cali'
datos_ideam_cali = 'IDEAM'
ruta_serie_cali_ideam= os.path.join(ruta_entrada,datos_cali,datos_ideam_cali+'.csv')
ideam_cali= pd.read_csv(ruta_serie_cali_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_cali = 'NREL'
ruta_serie_cali_nrel= os.path.join(ruta_entrada,datos_cali,datos_nrel_cali+'.csv')
nrel_cali= pd.read_csv(ruta_serie_cali_nrel, sep = ';',
                encoding='latin-1')

datos_barranquilla = 'Barranquilla'
datos_ideam_barranquilla = 'IDEAM'
ruta_serie_barranquilla_ideam= os.path.join(ruta_entrada,datos_barranquilla,datos_ideam_barranquilla+'.csv')
ideam_barranquilla= pd.read_csv(ruta_serie_barranquilla_ideam, sep = ';',
                encoding='latin-1')
datos_nrel_barranquilla = 'NREL'
ruta_serie_barranquilla_nrel= os.path.join(ruta_entrada,datos_barranquilla,datos_nrel_barranquilla+'.csv')
nrel_barranquilla= pd.read_csv(ruta_serie_barranquilla_nrel, sep = ';',
                encoding='latin-1')

## Punto 1

In [88]:
def GHIprom(serie_horaria):  #Entrada es una serie en resolución horaria, retorna DataFrame de promedios diarios mensuales
    agnos=sorted(serie_horaria['AÑO'].unique())
    ghi_df=pd.DataFrame(columns=["AÑO","MES","DIAS","GHImes"])
    energia = list(serie_horaria)[-1]
    for agno in agnos:
        for mes in range(1,13):
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == agno) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == agno)
                            & (serie_horaria["MES"] == mes)),
                                        energia].sum()/1000
            ghi_dict = {"AÑO":agno, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
        
        
    agnos2=sorted(ghi_df['AÑO'].unique())
    df_dias=pd.DataFrame(columns=["AÑO","MES","GHIdia"])
    for agno in agnos2:
        for mes in range(1,13):
            dia=ghi_df.loc[((ghi_df['AÑO']==agno) &
                                   (ghi_df['MES']==mes)),'DIAS'].item()
            ghi_diario=ghi_df.loc[((ghi_df['AÑO']==agno) & 
                                          (ghi_df['MES']==mes)),'GHImes'].item()/dia
            ghi_dict = {'AÑO':agno,'MES':mes,'GHIdia':ghi_diario}
            df_dias=df_dias.append(ghi_dict,ignore_index=True)
            
    return df_dias

def ENFICC(serie): #Calcula la ENFICC 99% del DataFrame determinado en la función anterior
    ENFICC=serie.quantile(0.01,numeric_only=True)['GHIdia']
    
    return print('La ENFICC 99% es', ENFICC)

## Punto 2

In [114]:
def graficas(serie_ideam,serie_corregida,serie_NREL): #Entra serie ideam, serie corregida y serie nrel con promedio diario mensual
    serie_ideam=sorted(serie_ideam["GHIdia"])
    serie_corregida=sorted(serie_corregida["GHIdia"])
    serie_NREL=sorted(serie_NREL["GHIdia"])
    x1=list(range(len(serie_ideam)))
    x2=list(range(len(serie_corregida)))
    x3=list(range(len(serie_NREL)))
    plt.plot(x1,serie_ideam/serie_ideam[0],'r',label='IDEAM')
    plt.plot(x2,serie_corregida/serie_corregida[-1],'b',label='corregida')
    plt.plot(x3,serie_NREL/serie_NREL[-1],'g',label='NREL')
    plt.xlabel('datos')
    plt.ylabel('Energía/Energía base ideam')
    plt.legend()
    return plt.show()

## Punto 3

In [10]:
def MCP(serie_corta,serie_larga): #Entra una serie de corto tiempo y una serie de referencia de duración completa.
    ghi_obj=serie_corta['IDEAM'].mean()
    sigma_obj=serie_corta['IDEAM'].std()
    serie_larga_comun=serie_larga[(serie_larga['AÑO']>=(serie_corta.iloc[0,0])) & (serie_larga['AÑO']<=(serie_corta.iloc[-1,0])) 
                                  &(serie_larga['MES']>=(serie_corta.iloc[0,1])) & (serie_larga['MES']<=(serie_corta.iloc[-1,1])) ]
    ghi_ref=serie_larga_comun['NREL'].mean()
    sigma_ref=serie_larga_comun['NREL'].std()
    ghi_i=(ghi_obj-(sigma_obj/sigma_ref)*ghi_ref)+(sigma_obj/sigma_ref)*serie_larga['NREL']
    ghi_i[ghi_i<0]=0
    serie_larga['NREL']=ghi_i
    return serie_larga #Retorna serie con datos GHI ajustados

## Punto 4

#### MBE

In [115]:
def MBE(serie_mod,serie_obs): #Entra serie corregida o nrel, y serie ideam
    ghi_mod=sorted(serie_mod['GHIdia'])
    ghi_obs=sorted(serie_obs['GHIdia'])
    ghi_mod=ghi_mod[:len(ghi_obs)]
    suma1=sum(ghi_mod)
    suma2=sum(ghi_obs)
    error=suma1-suma2
    resultado=error/suma2
    return print('El MBE es:',"{:.2%}".format(resultado) )

#### RMSE

In [12]:
def RMSE2(serie_mod,serie_obs): #Entra serie corregida o nrel, y serie ideam
    ghi_mod=sorted(serie_mod['GHIdia'])
    ghi_obs=sorted(serie_obs['GHIdia'])
    ghi_mod=ghi_mod[:len(ghi_obs)]
    suma1=sum(ghi_mod)
    suma2=sum(ghi_obs)
    ##PENDIENTE
    return print('El RMSE es:',"{:.2%}".format(resultado) )

#### PVALUE

In [13]:
def KSI(serie_mod,serie_obs):
    serie_mod2=serie_mod[(serie_mod['AÑO'] >=serie_obs.iloc[0,0]) &(serie_mod['AÑO']<=serie_obs.iloc[-1,0])]
    error=sp.stats.ks_2samp(serie_mod2['IDEAM'],serie_obs['NREL'])
    return print('El pvalue es:',error[-1])

# Análisis

In [117]:
corta=ideam_valledupar[(ideam_valledupar['AÑO']==2010)]
serie_corregida=MCP(corta,nrel_valledupar)
corregida_prom=GHIprom(serie_corregida)
ideam_prom=GHIprom(ideam_valledupar)
nrel_prom=GHIprom(nrel_valledupar)

In [ ]:
def RMSE(serie_mod,serie_obs): #Entra serie corregida o nrel, y serie ideam
    serie_mod2=serie_mod[(serie_mod['AÑO'] >=serie_obs.iloc[0,0]) &(serie_mod['AÑO']<=serie_obs.iloc[-1,0])]
    error=0
    for i in range(0,len(serie_obs)):
        resta=(serie_obs.iloc[i,4]-serie_mod2.iloc[i,4])**2
        error=error+resta
    error=math.sqrt(error/len(serie_obs))
    resultado=error/(serie_obs['IDEAM'][serie_obs['IDEAM'].idxmax()]-serie_obs['IDEAM'][serie_obs['IDEAM'].idxmin()])
    return print('El RMSE es:',"{:.2%}".format(resultado) )

In [118]:
MBE(corregida_prom,ideam_prom)

El MBE es: -2.55%
